In [1]:
import ipyparallel as ipp

c = ipp.Client(profile="mpi")
view = c[:]
view.activate()
view.block = True

import os
view["cwd"] = os.getcwd()
%px cd $cwd

[stdout:0] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks
[stdout:1] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks
[stdout:2] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks
[stdout:3] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks


In [2]:
import bodo

In [3]:
%%px
import bodo
import numpy as np
import time


@bodo.jit
def calc_pi(n):
    t1 = time.time()
    x = 2 * np.random.ranf(n) - 1
    y = 2 * np.random.ranf(n) - 1
    pi = 4 * np.sum(x**2 + y**2 < 1) / n
    print("Execution time:", time.time()-t1, "\nresult:", pi)

calc_pi(2 * 10**8)

[stdout:0] 
Execution time: 0.7505171298980713 
result: 3.14153352
